In [1]:
# USGS-15-CA-hesperia-e1902-s1898-rp1912: empty data, so not used
import glob
import pandas as pd

# read all input files and check column names
for filename in glob.glob("input/*.csv"):
    print(filename)
    print(pd.read_csv(filename).columns)
    print("========")

input\101201496.csv
Index(['id', 'TXT', 'PHRASE', 'TXT_LOC'], dtype='object')
input\102903919.csv
Index(['TXT', 'PHRASE', 'TXT_LOC', 'ID'], dtype='object')
input\102903922.csv
Index(['ID', 'TXT', 'PHRASE', 'TXT_LOC'], dtype='object')
input\102903925.csv
Index(['TXT', 'PHRASE', 'TXT_LOC', 'ID'], dtype='object')
input\102903928.csv
Index(['ID', 'TXT', 'PHRASE', 'TXT_LOC'], dtype='object')
input\103199059.csv
Index(['id', 'txt', 'phrase', 'txt_loc'], dtype='object')
input\103680995.csv
Index(['id', 'txt', 'phrase', 'txt_loc'], dtype='object')
input\103681073.csv
Index(['id', 'Name', 'Text', 'Position'], dtype='object')
input\103681079.csv
Index(['id', 'txt', 'phrase', 'txt_loc'], dtype='object')
input\103681115.csv
Index(['id', 'txt', 'phrase', 'txt_loc'], dtype='object')
input\USGS-15-CA-brawley-e1957-s1957-p1961.csv
Index(['id', 'txt', 'phrase', 'txt_loc'], dtype='object')
input\USGS-15-CA-capesanmartin-e1921-s1917.csv
Index(['id', 'txt', 'phrase', 'txt_loc'], dtype='object')
input\USGS

In [2]:
phrase_dic = dict()

for filename in glob.glob("input/*.csv"):
    key = filename[6:-3] +'jpg'
    try:
        try:
            phrase_dic[key] = set(pd.read_csv(filename).phrase)
        except:
            phrase_dic[key] = set(pd.read_csv(filename).PHRASE)        
    except:
        try:
            phrase_dic[key] = set(pd.read_csv(filename).Phrase)
        except:
            phrase_dic[key] = set(pd.read_csv(filename).Text)

In [3]:
phrase_dic

{'101201496.jpg': {'52•5',
  '52•9',
  '54•1',
  '54•5',
  '54•6',
  '54•9',
  '54•91',
  '55•2',
  '55•3',
  '55•4',
  '56•4',
  '56•9',
  '57•4',
  '58•0',
  '58•5',
  '59•7',
  '60•3',
  '61•0',
  '61•5',
  '62•1',
  '62•2',
  '63•3',
  '64•1',
  '65•1',
  '65•2',
  '65•9',
  '66•3',
  '68•0',
  '68•4',
  '70•4',
  '70•5',
  '70•8',
  '70•9',
  '73•3',
  '74•2',
  'ALBION STREET',
  'ALBION YARD',
  'ALDENHAM STREET',
  'ALPHA PLACE',
  'ARGYLE PLACE',
  'ARGYLE SQUARE',
  'ARGYLE STREET',
  'Air Shaft',
  'Albion Tin Works',
  'B.M. 51•39',
  'B.M. 54•87',
  'B.M. 55•62',
  'B.M. 56•91',
  'B.M. 57•90',
  'B.M. 58•67',
  'B.M. 58•98',
  'B.M. 59•11',
  'B.M. 63•41',
  'B.M. 65•73',
  'B.M. 66•10',
  'B.M. 66•92',
  'B.M. 67•16',
  'B.M. 67•45',
  'B.M. 67•85',
  'B.M. 68•28',
  'B.M. 68•88',
  'B.M. 69•81',
  'B.M. 70•04',
  'B.M. 70•16',
  'B.M. 71•29',
  'B.M. 71•40',
  'B.M. 72•24',
  'B.M. 74•22',
  'B.M. 85•73',
  'B.O.',
  'B.P',
  'B.S',
  'BATTLE BRIDGE ROAD',
  'BELGRAVE S

In [4]:
# the number of distinct phrase per each map
for key in phrase_dic:
    print(len(phrase_dic[key]))

242
105
89
87
77
123
71
38
41
43
166
142
118
167
191
49
123
240
280
106
131
131
84
160


In [5]:
from rdflib import Graph, URIRef, Literal, XSD, Namespace
from rdflib.namespace import RDF

In [6]:
# http://www.ldf.fi/service/rdf-grapher

MYNS = Namespace('gs:kg:')
schema = Namespace('http://schema.org/')
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
geo = Namespace("http://www.opengis.net/ont/geosparql#")
geoname = Namespace("http://linkedgeodata.org/ontology/")

my_kg = Graph()
my_kg.bind('my_ns', MYNS)
my_kg.bind('schema', schema)
my_kg.bind('geo', geo)
my_kg.bind('rdfs', rdfs)
my_kg.bind('geoname', geoname)

In [7]:
for key in phrase_dic:
    map_name = key
    node_uri = URIRef(MYNS["mapName:"+map_name])
    my_kg.add((node_uri, rdfs['type'], schema['Map']))
    my_kg.add((node_uri, geoname['nearby'], Literal(key+": Name nearby:addLater")))
    
    i = 0
    for phrase in phrase_dic[key]:
        i += 1
        feature_uri = URIRef(MYNS[key+'feature_id'+str(i)])
        my_kg.add((node_uri, geo['sfOverlaps'], feature_uri))

        my_kg.add((feature_uri, rdfs['label'], Literal(phrase)))
        my_kg.add((feature_uri, rdfs['seeAlso'], URIRef(key+"externalDBuri"+str(i))))

        my_kg.add((feature_uri, geo['asWKT'], Literal(key+": addLater"+str(i))))


In [8]:
my_kg.serialize('mapRDF.ttl', format="turtle")